In [ ]:
from tensorflow.keras.models import load_model


cnn_model = load_model('/content/model.h5')
cnn_3d_model = load_model('/content/3d_cnn_deepfake_detection.h5')
efficientnet_model = load_model('/content/detection.h5')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import cv2


test_dir = '/content/drive/MyDrive/crop_face/faces'
real_test_dir = os.path.join(test_dir, 'real_video')
fake_test_dir = os.path.join(test_dir, 'fake_video')

def load_test_images(image_dir, target_size=(128, 128)):
    image_paths = []
    labels = []
    for label, folder_name in [(0, 'real_video'), (1, 'fake_video')]:
        folder_path = os.path.join(image_dir, folder_name)
        # Loop through each subdirectory
        for subdir in os.listdir(folder_path):
            subdir_path = os.path.join(folder_path, subdir)
            if os.path.isdir(subdir_path):
                for img_file in os.listdir(subdir_path):
                    if img_file.endswith('.jpg'):
                        img_path = os.path.join(subdir_path, img_file)
                        image_paths.append(img_path)
                        labels.append(label)

    # Load images and resize
    images = [cv2.resize(cv2.imread(path), target_size).astype('float32') / 255.0 for path in image_paths]
    return np.array(images), np.array(labels)

# Load test images
X_test, y_test = load_test_images(test_dir)


In [ ]:
# Evaluate the CNN model on the test set
cnn_loss, cnn_acc = cnn_model.evaluate(X_test, y_test, verbose=1)
print(f"CNN Model - Test Loss: {cnn_loss}, Test Accuracy: {cnn_acc}")


88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9614 - loss: 0.0967
CNN Model - Test Loss: 0.08062057942152023, Test Accuracy: 0.9668921232223511


In [ ]:
import os
import numpy as np
import cv2

def load_test_sequences(base_dir, sequence_length, target_size=(64, 64)):
    face_sequences = []
    labels = []


    for video_type, label_value in [('real_video', 0), ('fake_video', 1)]:
        video_dir = os.path.join(base_dir, video_type)
        if not os.path.exists(video_dir):
            continue


        for subdir in os.listdir(video_dir):
            subdir_path = os.path.join(video_dir, subdir)
            if not os.path.isdir(subdir_path):
                continue


            face_files = sorted([f for f in os.listdir(subdir_path) if f.endswith('.jpg')])


            num_sequences = len(face_files) - sequence_length + 1
            # Ensure that the number of sequences matches the number of labels
            # by taking only non-overlapping sequences
            for i in range(0, num_sequences, sequence_length):
                sequence = face_files[i:i + sequence_length]

                # Check if the sequence has enough frames
                if len(sequence) == sequence_length:
                    face_sequences.append([os.path.join(subdir_path, frame) for frame in sequence])
                    labels.append(label_value)


    X_test1 = []
    for sequence in face_sequences:
        frames = [cv2.resize(cv2.imread(frame), target_size).astype('float32') / 255.0 for frame in sequence]
        X_test1.append(np.stack(frames))

    return np.array(X_test1), np.array(labels) # Return X_test1 instead of X_test

# Set test data directory and sequence length
sequence_length = 10
X_test1, y_test1 = load_test_sequences(test_dir, sequence_length)

In [ ]:
X_test1.shape

(268, 10, 64, 64, 3)

In [ ]:
y_test1.shape

(268,)

In [ ]:
# Evaluate the 3D CNN model on the test set
cnn_3d_loss, cnn_3d_acc = cnn_3d_model.evaluate(X_test1, y_test1, verbose=1)
print(f"3D CNN Model - Test Loss: {cnn_3d_loss}, Test Accuracy: {cnn_3d_acc}")


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.5805 - loss: 0.6970
3D CNN Model - Test Loss: 0.7007034420967102, Test Accuracy: 0.60447758436203


In [ ]:
# Evaluate the EfficientNet model on the test set
efficientnet_loss, efficientnet_acc = efficientnet_model.evaluate(X_test, y_test, verbose=1)
print(f"EfficientNet Model - Test Loss: {efficientnet_loss}, Test Accuracy: {efficientnet_acc}")


88/88 ━━━━━━━━━━━━━━━━━━━━ 15s 69ms/step - accuracy: 0.0924 - loss: 0.6961
EfficientNet Model - Test Loss: 0.6939706802368164, Test Accuracy: 0.3851904571056366


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Get predictions
y_pred = np.round(cnn_model.predict(X_test)).astype(int)

# Print classification report
print(classification_report(y_test, y_pred))

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)


88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
              precision    recall  f1-score   support

           0       0.99      0.96      0.97      1727
           1       0.94      0.98      0.96      1082

    accuracy                           0.97      2809
   macro avg       0.96      0.97      0.97      2809
weighted avg       0.97      0.97      0.97      2809

[[1658   69]
 [  24 1058]]


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained 3D CNN model
model = load_model('3d_cnn_deepfake_detection.h5')

# Define a function to extract frames from the video
def extract_frames_from_video(video_path, target_size=(64, 64)):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame
        resized_frame = cv2.resize(frame, target_size)
        normalized_frame = resized_frame.astype('float32') / 255.0  # Normalize to [0,1]

        frames.append(normalized_frame)

    cap.release()
    return np.array(frames)

# Define a function to create sequences of frames
def create_frame_sequences(frames, sequence_length):
    sequences = []
    for i in range(0, len(frames) - sequence_length + 1):
        sequence = frames[i:i + sequence_length]
        sequences.append(sequence)
    return np.array(sequences)

# Path to the video file
video_path = '/content/00010.mp4'

# Extract frames from the video
frames = extract_frames_from_video(video_path)

# Check if the number of frames is less than the required sequence length
sequence_length = 10  # This should match what the 3D CNN model was trained on
if len(frames) < sequence_length:
    print(f"Video is too short! Needs at least {sequence_length} frames.")
else:
    # Create sequences from the frames
    frame_sequences = create_frame_sequences(frames, sequence_length)

    # Predict using the 3D CNN model
    predictions = model.predict(frame_sequences)

    # Average the predictions across all sequences
    avg_prediction = np.mean(predictions)

    # Output the result
    if avg_prediction > 0.5:
        print(f"Prediction: Deepfake (Confidence: {avg_prediction * 100:.2f}%)")
    else:
        print(f"Prediction: Real (Confidence: {(1 - avg_prediction) * 100:.2f}%)")


21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step
Prediction: Deepfake (Confidence: 100.00%)


In [ ]:
# Path to the video file
video_path = '/content/00096.mp4'

# Extract frames from the video
frames = extract_frames_from_video(video_path)

# Check if the number of frames is less than the required sequence length
sequence_length = 10  # This should match what the 3D CNN model was trained on
if len(frames) < sequence_length:
    print(f"Video is too short! Needs at least {sequence_length} frames.")
else:
    # Create sequences from the frames
    frame_sequences = create_frame_sequences(frames, sequence_length)

    # Predict using the 3D CNN model
    predictions = model.predict(frame_sequences)

    # Average the predictions across all sequences
    avg_prediction = np.mean(predictions)

    # Output the result
    if avg_prediction > 0.5:
        print(f"Prediction: Deepfake (Confidence: {avg_prediction * 100:.2f}%)")
    else:
        print(f"Prediction: Real (Confidence: {(1 - avg_prediction) * 100:.2f}%)")


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step
Prediction: Real (Confidence: 100.00%)


In [ ]:
# Path to the video file
video_path = '/content/00131.mp4'

# Extract frames from the video
frames = extract_frames_from_video(video_path)

# Check if the number of frames is less than the required sequence length
sequence_length = 10  # This should match what the 3D CNN model was trained on
if len(frames) < sequence_length:
    print(f"Video is too short! Needs at least {sequence_length} frames.")
else:
    # Create sequences from the frames
    frame_sequences = create_frame_sequences(frames, sequence_length)

    # Predict using the 3D CNN model
    predictions = model.predict(frame_sequences)

    # Average the predictions across all sequences
    avg_prediction = np.mean(predictions)

    # Output the result
    if avg_prediction > 0.5:
        print(f"Prediction: Deepfake (Confidence: {avg_prediction * 100:.2f}%)")
    else:
        print(f"Prediction: Real (Confidence: {(1 - avg_prediction) * 100:.2f}%)")

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step
Prediction: Real (Confidence: 55.14%)


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained 2D CNN model
model = load_model('model.h5')  # Assuming you saved your 2D CNN as 'model.h5'

# Define a function to extract frames from the video
def extract_frames_from_video(video_path, target_size=(128, 128)):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame to match model input size
        resized_frame = cv2.resize(frame, target_size)
        normalized_frame = resized_frame.astype('float32') / 255.0  # Normalize to [0,1]

        frames.append(normalized_frame)

    cap.release()
    return np.array(frames)

# Path to the video file
video_path = '/content/00131.mp4'

# Extract frames from the video
frames = extract_frames_from_video(video_path)

# Predict each frame using the 2D CNN model
predictions = model.predict(frames)

# Average the predictions across all frames
avg_prediction = np.mean(predictions)

# Output the result
if avg_prediction > 0.5:
    print(f"Prediction: Deepfake ")
else:
    print(f"Prediction: Real ")


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Prediction: Real 


In [ ]:
# Path to the video file
video_path = '/content/00519.mp4'

# Extract frames from the video
frames = extract_frames_from_video(video_path)

# Predict each frame using the 2D CNN model
predictions = model.predict(frames)

# Average the predictions across all frames
avg_prediction = np.mean(predictions)

# Output the result
if avg_prediction > 0.5:
    print(f"Prediction: Deepfake ")
else:
    print(f"Prediction: Real ")


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step
Prediction: Deepfake 


In [ ]:

video_path = '/content/02480.mp4'

# Extract frames from the video
frames = extract_frames_from_video(video_path)

# Predict each frame using the 2D CNN model
predictions = model.predict(frames)

# Average the predictions across all frames
avg_prediction = np.mean(predictions)

# Output the result
if avg_prediction > 0.5:
    print(f"Prediction: Deepfake ")
else:
    print(f"Prediction: Real ")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Prediction: Deepfake 
